# Recommender

In [1]:
import pandas as pd
import os
# import datetime as dt

#### Import Data

In [2]:
df = pd.read_csv("data{}KaDo_Timestamps.csv".format(os.sep), sep=",")

df.head()

,TICKET_ID,MOIS_VENTE,PRIX_NET,FAMILLE,UNIVERS,MAILLE,LIBELLE,CLI_ID,timestamps
0,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,1.633085e+09
1,35592159,10,1.66,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,1.633085e+09
2,35592159,10,7.45,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JR PARF BIO.SPE AC.SENT.50ML,1490281,1.633085e+09
3,35592159,10,5.95,SOINS DU VISAGE,VIS_DEMAQ AAAR,VIS_AAAR_DEMAQLOTION,EAU MICELLAIRE 3 THES FL200ML,1490281,1.633085e+09
4,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 TIARE FL 200ML,1490281,1.633085e+09


In [15]:
df.shape

(7245522, 9)

In [3]:
df = df.drop_duplicates(subset=["FAMILLE", "UNIVERS", "MAILLE", "LIBELLE", "TICKET_ID"]).reset_index(drop=True)

df.head()

,TICKET_ID,MOIS_VENTE,PRIX_NET,FAMILLE,UNIVERS,MAILLE,LIBELLE,CLI_ID,timestamps
0,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,1.633085e+09
1,35592159,10,7.45,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JR PARF BIO.SPE AC.SENT.50ML,1490281,1.633085e+09
2,35592159,10,5.95,SOINS DU VISAGE,VIS_DEMAQ AAAR,VIS_AAAR_DEMAQLOTION,EAU MICELLAIRE 3 THES FL200ML,1490281,1.633085e+09
3,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 TIARE FL 200ML,1490281,1.633085e+09
4,35509899,9,13.00,PARFUMAGE,PARF_EAUX PARFUMS,PARF_PARFUM,EDT UN MATIN AU JARDIN 100ML MUGUET,13290776,1.630493e+09


In [17]:
df.shape

(6955521, 9)

#### Define categories for categorization

In [4]:
CATEGORIES = []

CATEGORIES.extend(df["FAMILLE"].unique().tolist())
CATEGORIES.extend(df["MAILLE"].unique().tolist())
CATEGORIES.extend(df["UNIVERS"].unique().tolist())
CATEGORIES.extend(df["TICKET_ID"].unique().tolist())
CATEGORIES.extend(df["LIBELLE"].unique().tolist())

CATEGORIES = list(dict.fromkeys(CATEGORIES))

CATEGORIES[:5]

['HYGIENE', 'SOINS DU VISAGE', 'PARFUMAGE', 'SOINS DU CORPS', 'MAQUILLAGE']

#### Categorization of string to convert them to int

In [6]:
df["ITEM_ID"] = pd.Categorical(df["LIBELLE"], categories=CATEGORIES)
df["ITEM_ID"] = df["ITEM_ID"].cat.codes

df.head()

,TICKET_ID,MOIS_VENTE,PRIX_NET,FAMILLE,UNIVERS,MAILLE,LIBELLE,CLI_ID,timestamps,ITEM_ID
0,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,1.633085e+09,2734987
1,35592159,10,7.45,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JR PARF BIO.SPE AC.SENT.50ML,1490281,1.633085e+09,2734988
2,35592159,10,5.95,SOINS DU VISAGE,VIS_DEMAQ AAAR,VIS_AAAR_DEMAQLOTION,EAU MICELLAIRE 3 THES FL200ML,1490281,1.633085e+09,2734989
3,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 TIARE FL 200ML,1490281,1.633085e+09,2734990
4,35509899,9,13.00,PARFUMAGE,PARF_EAUX PARFUMS,PARF_PARFUM,EDT UN MATIN AU JARDIN 100ML MUGUET,13290776,1.630493e+09,2734991


# SPOTLIGHT

## Create Interactions Matrice

In [6]:
from spotlight.interactions import Interactions

implicit_interactions = Interactions(df["CLI_ID"].values, df["ITEM_ID"].values, timestamps=df["timestamps"])

In [10]:
from spotlight.sequence.implicit import ImplicitSequenceModel

model = ImplicitSequenceModel(representation='cnn')

interactions = implicit_interactions.to_sequence()

## Train Model

In [11]:
model.fit(interactions, verbose=True)

Epoch 0: loss 0.1742262915851945
Epoch 1: loss 0.2234868087659179
Epoch 2: loss 0.28834821103129077
Epoch 3: loss 0.2649398190038563
Epoch 4: loss 0.2565682430623447
Epoch 5: loss 0.2766829534463268
Epoch 6: loss 0.23482881989005103
Epoch 7: loss 0.15953951578843242
Epoch 8: loss 0.12515916047413597
Epoch 9: loss 0.12309346634163673


## Save Model

In [12]:
import pickle
import os

pickle.dump(model, open("models{}model_spotlight_dup_mix.pkl".format(os.sep), "wb"))